In [10]:
import pandas as pd
from imblearn.over_sampling import SMOTE, SMOTEN
from sklearn.metrics import confusion_matrix, precision_score, recall_score, auc, roc_auc_score, average_precision_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.neural_network import MLPClassifier
import numpy as np
from copy import deepcopy
import functools
import networkx as nx
from hiclass import BinaryPolicy
from hiclass.ConstantClassifier import ConstantClassifier
from hiclass.MultiLabelHierarchicalClassifier import MultiLabelHierarchicalClassifier, make_leveled
from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_is_fitted
from hiclass import LocalClassifierPerNode
from hiclass.MultiLabelLocalClassifierPerNode import MultiLabelLocalClassifierPerNode
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import plotly.express as px
import sklearn


- X: embeddings large
- y: ppa (single)

Explode ppa

Evaluation:  per ppa  (AUC & PR)

In [11]:
outputs_df = pd.read_parquet("../data/processed/outputs_openai_embeddings_v1.parquet")
ppas = outputs_df.PPAs_list.explode().drop_duplicates().sort_values().to_list()
train_df = outputs_df[~outputs_df.test_set]
test_df = outputs_df[outputs_df.test_set]
X_labels = outputs_df.columns[outputs_df.columns.str.contains("openai_embedding_small")]
# y_label = "PPAs_list"
oversampling = True

X_train = train_df[X_labels].to_numpy()
# y_train = train_df[y_label].to_numpy()
y_train = train_df[ppas].astype(int).to_numpy()

X_test = test_df[X_labels].to_numpy()
# y_test = test_df[y_label].to_numpy()
y_test = test_df[ppas].astype(int).to_numpy()




In [12]:


# if oversampling:
#     ros = RandomOverSampler(random_state=42)
#     X_train, y_train = ros.fit_resample(X_train, y_train)




- X: embeddings large
- y: ppa (single)

Explode ppa

Evaluation:  per ppa  (AUC & PR)

In [13]:


# if oversampling:
#     ros = RandomOverSampler(random_state=42)
#     X_train, y_train = ros.fit_resample(X_train, y_train)

if oversampling:
    train_oversamples_df, _ = ros.fit_resample(train_df, y=train_df["primary_ppa"])
    X_train = train_df[X_labels].to_numpy()
# y_train = train_df[y_label].to_numpy()
    y_train = train_df[ppas].astype(int).to_numpy() 

In [ ]:

from sklearn.linear_model import RidgeClassifier


clf = RidgeClassifier(random_state=99)
# clf = LogisticRegression(random_state=99, max_iter=10_000)
clf.fit(X_train, y_train)
# random_forest.score(X_test, y_test)

# y_pred = clf.predict(X_test)
# y_pred_proba = clf.predict_proba(X_test)

d = clf.decision_function(X_test)
print(d.shape)
y_pred_proba = np.exp(d) / np.sum(np.exp(d))
print(y_pred_proba.shape)
y_pred_proba_df = pd.DataFrame(y_pred_proba, columns=ppas)
# y_pred_proba_df = pd.DataFrame(np.array(y_pred_proba)[:, :, 0].T, columns=ppas)

experiment = []
for ppa in ppas:    
    y_test_ppa = test_df[ppa].astype(int)
    experiment.append({
        "ppa": ppa,
        "roc_auc": roc_auc_score(y_test_ppa, y_pred_proba_df[ppa]),
        "average_precision": average_precision_score(y_test_ppa, y_pred_proba_df[ppa])
    })
    
experiment_df = pd.DataFrame(experiment)
experiment_df.describe()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier


clf = KNeighborsClassifier()
# clf = LogisticRegression(random_state=99, max_iter=10_000)
clf.fit(X_train, y_train)
# random_forest.score(X_test, y_test)

# y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)

# d = clf.decision_function(X_test)
# print(d.shape)
# y_pred_proba = np.exp(d) / np.sum(np.exp(d))
# print(y_pred_proba.shape)
# y_pred_proba_df = pd.DataFrame(y_pred_proba, columns=ppas)
y_pred_proba_df = pd.DataFrame(np.array(y_pred_proba)[:, :, 0].T, columns=ppas)

experiment = []
for ppa in ppas:    
    y_test_ppa = test_df[ppa].astype(int)
    experiment.append({
        "ppa": ppa,
        "roc_auc": roc_auc_score(y_test_ppa, y_pred_proba_df[ppa]),
        "average_precision": average_precision_score(y_test_ppa, y_pred_proba_df[ppa])
    })
    
experiment_df = pd.DataFrame(experiment)
experiment_df.describe()

In [14]:

clf = MLPClassifier(random_state=99, max_iter=10_000)
# clf = LogisticRegression(random_state=99, max_iter=10_000)
clf.fit(X_train, y_train)
# random_forest.score(X_test, y_test)

# y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)
y_pred_proba_df = pd.DataFrame(y_pred_proba, columns=ppas)
# y_pred_proba_df = pd.DataFrame(np.array(y_pred_proba)[:, :, 0].T, columns=ppas)

experiment = []
for ppa in ppas:    
    y_test_ppa = test_df[ppa].astype(int)
    experiment.append({
        "ppa": ppa,
        "roc_auc": roc_auc_score(y_test_ppa, y_pred_proba_df[ppa]),
        "average_precision": average_precision_score(y_test_ppa, y_pred_proba_df[ppa])
    })
    
experiment_df = pd.DataFrame(experiment)
experiment_df.describe()

,roc_auc,average_precision
count,20.000000,20.000000
mean,0.881520,0.556749
std,0.136083,0.282911
min,0.537143,0.083916
25%,0.837445,0.291171
50%,0.927083,0.597222
75%,0.972973,0.750000
max,1.000000,1.000000


In [ ]:
px.scatter(pd.DataFrame(experiment), x="roc_auc", y="average_precision", text="ppa").show()

- logistic regression OvA
- hierarquical
- con / sin oversampling
- large/small embeddings
- 


In [5]:
y_train = train_df.PPAs_list.apply(lambda ppa_list: [[ppa[:2], ppa] for ppa in ppa_list]).to_list()
y_test = test_df.PPAs_list.apply(lambda ppa_list: [[ppa[:2], ppa] for ppa in ppa_list]).to_list()

y_train = np.asarray(y_train, dtype="object")
y_test = np.asarray(y_test, dtype="object")

In [6]:
def predict1(self, X, tolerance: float = None) -> np.ndarray:
    r"""
    Predict classes for the given data.

    Hierarchical labels are returned.

    Parameters
    ----------
    X : {array-like, sparse matrix} of shape (n_samples, n_features)
        The input samples. Internally, its dtype will be converted
        to ``dtype=np.float32``. If a sparse matrix is provided, it will be
        converted into a sparse ``csr_matrix``.
    tolerance : float, default=None
        The tolerance used to determine multi-labels.
        If set to None, only the child class with highest probability is predicted.
        Overrides the tolerance set in the constructor.
        Otherwise, all child classes with :math:`probability >= max\_prob - tolerance` are predicted.
    Returns
    -------
    y : ndarray of shape (n_samples,) or (n_samples, n_outputs)
        The predicted classes.
    """
    # Check if fit has been called
    check_is_fitted(self)
    _tolerance = (tolerance if tolerance is not None else self.tolerance) or 0.0

    # Input validation
    if not self.bert:
        X = sklearn.utils.validation.check_array(
            X, accept_sparse="csr"
        )  # TODO: Decide allow_nd True or False
    else:
        X = np.array(X)

    # Initialize array that holds predictions
    y = [[[]] for _ in range(X.shape[0])]

    bfs = nx.bfs_successors(self.hierarchy_, source=self.root_)

    self.logger_.info("Predicting")

    probas = []
    for predecessor, successors in bfs:
        if predecessor == self.root_:
            mask = [True] * X.shape[0]
            subset_x = X[mask]
            y_row_indices = [[i, [0]] for i in range(X.shape[0])]
        else:
            # get indices of rows that have the predecessor
            y_row_indices = []
            for row in range(X.shape[0]):
                # create list of indices
                _t = [z for z, ls in enumerate(y[row]) if ls[-1] == predecessor]

                # y_row_indices is a list of lists, each list contains the index of the row and a list of column indices
                y_row_indices.append([row, _t])

            # Filter
            mask = [True if ld[1] else False for ld in y_row_indices]
            y_row_indices = [k for k in y_row_indices if k[1]]
            subset_x = X[mask]

        if subset_x.shape[0] > 0:
            probabilities = np.zeros((subset_x.shape[0], len(successors)))
            for row, successor in enumerate(successors):
                successor_name = str(successor).split(self.separator_)[-1]
                self.logger_.info(f"Predicting for node '{successor_name}'")
                classifier = self.hierarchy_.nodes[successor]["classifier"]
                positive_index = np.where(classifier.classes_ == 1)[0]


                # if hasattr(classifier, "predict_proba"):
                probabilities[:, row] = classifier.predict_proba(subset_x)[
                    :, positive_index
                ][:, 0]
                # else:
                #     print(subset_x.shape)
                #     d = classifier.decision_function(subset_x)
                #     y_pred_proba = np.exp(d) / np.sum(np.exp(d))
                #     print(y_pred_proba)
                #     probabilities[:, row] = y_pred_proba[
                #         :, positive_index
                #     ][:, 0]



            probas.append(pd.DataFrame(probabilities, columns=successors, index=[x0 for x0, x1 in y_row_indices]))

            # get indices of probabilities that are within tolerance of max

            highest_probabilities = np.max(probabilities, axis=1).reshape(-1, 1)

            # indices_probabilities_within_tolerance = np.argwhere(
            #     np.greater_equal(probabilities, highest_probabilities - _tolerance)
            # )
            indices_probabilities_within_tolerance = np.argwhere(
                np.greater_equal(probabilities, 0)
            )

            prediction = [[] for _ in range(subset_x.shape[0])]
            for row, column in indices_probabilities_within_tolerance:
                prediction[row].append(successors[column])

            k = 0  # index of prediction
            for row, col_list in y_row_indices:
                for j in col_list:
                    if not prediction[k]:
                        y[row][j].append("")
                    else:
                        for pi, _suc in enumerate(prediction[k]):
                            if pi == 0:
                                y[row][j].append(_suc)
                            else:
                                # in case of mulitple predictions, copy the previous prediction up to (but not including) the last prediction and add the new one
                                _old_y = y[row][j][:-1].copy()
                                y[row].insert(j + 1, _old_y + [_suc])
                k += 1

    y = make_leveled(y)
    self._remove_separator(y)
    y = np.array(y, dtype=self.dtype_)
    return probas
    return y

In [9]:
model = MLPClassifier(random_state=99, max_iter=10_000)
classifier = MultiLabelLocalClassifierPerNode(local_classifier=model)
classifier.fit(X_train, y_train)
# predictions = classifier.predict(X_test)

predictions = predict1(classifier, X_test)
xx = pd.concat(predictions, axis=1).fillna(0)
xx.columns = xx.columns.str[-4:]
# predictions[0].assign(better=lambda _df: _df.apply(lambda _row: _row.idxmax(), axis=1))
xx2 = xx.copy()
for ppa in ppas:
    xx2[ppa] = xx2.apply(lambda _row: _row[ppa]*_row[ppa[:2]], axis=1)
    
experiment = []
for ppa in ppas:    
    y_test_ppa = test_df[ppa].astype(int)
    experiment.append({
        "ppa": ppa,
        "roc_auc": roc_auc_score(y_test_ppa, xx2[ppa]),
        "average_precision": average_precision_score(y_test_ppa, xx2[ppa])
    })
experiment_df = pd.DataFrame(experiment)
experiment_df.describe()

,roc_auc,average_precision
count,20.000000,20.000000
mean,0.869147,0.628587
std,0.196130,0.258902
min,0.171053,0.044700
25%,0.854167,0.557738
50%,0.933499,0.631944
75%,0.975759,0.775000
max,1.000000,1.000000


In [16]:
px.scatter(pd.DataFrame(experiment), x="roc_auc", y="average_precision", text="ppa").show()

In [ ]:
model = RidgeClassifier(random_state=99)

def pred_prob(X_test, model1):
    d = model1.decision_function(X_test)
    # print(d.shape)
    y_pred_proba = np.exp(d) / np.sum(np.exp(d))
    return y_pred_proba
    # y_pred_proba_df = pd.DataFrame(y_pred_proba, columns=ppas)
    
# model.predict_proba = pred_prob

classifier = MultiLabelLocalClassifierPerNode(local_classifier=model)
classifier.fit(X_train, y_train)
# predictions = classifier.predict(X_test)

predictions = predict1(classifier, X_test)
xx = pd.concat(predictions, axis=1).fillna(0)
xx.columns = xx.columns.str[-4:]
# predictions[0].assign(better=lambda _df: _df.apply(lambda _row: _row.idxmax(), axis=1))
xx2 = xx.copy()
for ppa in ppas:
    xx2[ppa] = xx2.apply(lambda _row: _row[ppa]*_row[ppa[:2]], axis=1)
    
experiment = []
for ppa in ppas:    
    y_test_ppa = test_df[ppa].astype(int)
    experiment.append({
        "ppa": ppa,
        "roc_auc": roc_auc_score(y_test_ppa, xx2[ppa]),
        "average_precision": average_precision_score(y_test_ppa, xx2[ppa])
    })
experiment_df = pd.DataFrame(experiment)
experiment_df.describe()

In [ ]:

clf = RandomForestClassifier(random_state=99)
# clf = LogisticRegression(random_state=99, max_iter=10_000)
clf.fit(X_train, y_train)
# random_forest.score(X_test, y_test)

# y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)
# y_pred_proba_df = pd.DataFrame(y_pred_proba, columns=ppas)
y_pred_proba_df = pd.DataFrame(np.array(y_pred_proba)[:, :, 0].T, columns=ppas)

experiment = []
for ppa in ppas:    
    y_test_ppa = test_df[ppa].astype(int)
    experiment.append({
        "ppa": ppa,
        "roc_auc": roc_auc_score(y_test_ppa, y_pred_proba_df[ppa]),
        "average_precision": average_precision_score(y_test_ppa, y_pred_proba_df[ppa])
    })
    
experiment_df = pd.DataFrame(experiment)
experiment_df.describe()

In [ ]:

clf = MLPClassifier(random_state=99, max_iter=10_000)
# clf = LogisticRegression(random_state=99, max_iter=10_000)
clf.fit(X_train, y_train)
# random_forest.score(X_test, y_test)

# y_pred = clf.predict(X_test)
y_pred_proba = clf.predict_proba(X_test)
y_pred_proba_df = pd.DataFrame(y_pred_proba, columns=ppas)
# y_pred_proba_df = pd.DataFrame(np.array(y_pred_proba)[:, :, 0].T, columns=ppas)

experiment = []
for ppa in ppas:    
    y_test_ppa = test_df[ppa].astype(int)
    experiment.append({
        "ppa": ppa,
        "roc_auc": roc_auc_score(y_test_ppa, y_pred_proba_df[ppa]),
        "average_precision": average_precision_score(y_test_ppa, y_pred_proba_df[ppa])
    })
    
experiment_df = pd.DataFrame(experiment)
experiment_df.describe()

In [ ]:
px.scatter(pd.DataFrame(experiment), x="roc_auc", y="average_precision", text="ppa").show()

- logistic regression OvA
- hierarquical
- con / sin oversampling
- large/small embeddings
- 


# hierarquical

In [ ]:
y_train = train_df.PPAs_list.apply(lambda ppa_list: [[ppa[:2], ppa] for ppa in ppa_list]).to_list()
y_test = test_df.PPAs_list.apply(lambda ppa_list: [[ppa[:2], ppa] for ppa in ppa_list]).to_list()

y_train = np.asarray(y_train, dtype="object")
y_test = np.asarray(y_test, dtype="object")

In [ ]:
model = LassoCl(random_state=99)
classifier = MultiLabelLocalClassifierPerNode(local_classifier=model)
classifier.fit(X_train, y_train)
# predictions = classifier.predict(X_test)

predictions = predict1(classifier, X_test)
xx = pd.concat(predictions, axis=1).fillna(0)
xx.columns = xx.columns.str[-4:]
# predictions[0].assign(better=lambda _df: _df.apply(lambda _row: _row.idxmax(), axis=1))
xx2 = xx.copy()
for ppa in ppas:
    xx2[ppa] = xx2.apply(lambda _row: _row[ppa]*_row[ppa[:2]], axis=1)
    
experiment = []
for ppa in ppas:    
    y_test_ppa = test_df[ppa].astype(int)
    experiment.append({
        "ppa": ppa,
        "roc_auc": roc_auc_score(y_test_ppa, xx2[ppa]),
        "average_precision": average_precision_score(y_test_ppa, xx2[ppa])
    })
experiment_df = pd.DataFrame(experiment)
experiment_df.describe()

In [ ]:
model = LogisticRegression(random_state=99)
classifier = MultiLabelLocalClassifierPerNode(local_classifier=model)
classifier.fit(X_train, y_train)
# predictions = classifier.predict(X_test)

predictions = predict1(classifier, X_test)
xx = pd.concat(predictions, axis=1).fillna(0)
xx.columns = xx.columns.str[-4:]
# predictions[0].assign(better=lambda _df: _df.apply(lambda _row: _row.idxmax(), axis=1))
xx2 = xx.copy()
for ppa in ppas:
    xx2[ppa] = xx2.apply(lambda _row: _row[ppa]*_row[ppa[:2]], axis=1)
    
experiment = []
for ppa in ppas:    
    y_test_ppa = test_df[ppa].astype(int)
    experiment.append({
        "ppa": ppa,
        "roc_auc": roc_auc_score(y_test_ppa, xx2[ppa]),
        "average_precision": average_precision_score(y_test_ppa, xx2[ppa])
    })
experiment_df = pd.DataFrame(experiment)
experiment_df.describe()

In [ ]:
model = MLPClassifier(random_state=99, max_iter=10_000, hidden_layer_sizes=30)
classifier = MultiLabelLocalClassifierPerNode(local_classifier=model)
classifier.fit(X_train, y_train)
predictions = classifier.predict(X_test)

